In [10]:
%load_ext autoreload
# %autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import logging
from sys import stdout
from datetime import datetime
import numpy as np
import os

logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(levelname)s] %(message)s",
    stream=stdout,
    datefmt="%m-%d %H:%M:%S",
)

logging.info(f"{os.getlogin()}| {datetime.now()}")

[02-06 12:01:25][INFO] DELL| 2025-02-06 12:01:25.287735


In [19]:
def gauss_jordan(Ab: np.ndarray) -> np.ndarray:
    Ab = Ab.astype(float)
    m, n_plus_1 = Ab.shape
    n = n_plus_1 - 1

    for i in range(n):
        # Encontrar y verificar el pivote en la posición (i, i)
        pivot = Ab[i, i]
        if np.isclose(pivot, 0):
            raise ValueError("Pivote cercano a cero en la fila {}".format(i))
        Ab[i, :] = Ab[i, :] / pivot
        for j in range(m):
            if j != i:
                factor = Ab[j, i]
                Ab[j, :] = Ab[j, :] - factor * Ab[i, :]

    return Ab[:, -1].copy()

In [42]:
%autoreload 2
from src import matriz_aumentada

A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
b = [1, -2, 3, 4]

Ab = matriz_aumentada(A, b)
gauss_jordan(Ab)

array([ 3., -6.,  6., -2.])

# Resolver

In [50]:
def inv_matrix(A: np.ndarray) -> np.ndarray:
    A = np.array(A, dtype=float)
    n = A.shape[0]
    A_inv = np.zeros((n, n), dtype=float)
    for i in range(n):
        b = np.eye(n, dtype=float)[:, i]
        Aug = matriz_aumentada(A, b)
        
        for j in range(n):
            max_row = j + np.argmax(np.abs(Aug[j:, j]))
            if np.isclose(Aug[max_row, j], 0):
                raise ValueError(f"No se puede invertir: pivote cero en columna {j}.")
            # Intercambiar filas si es necesario
            if max_row != j:
                Aug[[j, max_row]] = Aug[[max_row, j]]
            
            # Normalizar la fila pivote
            pivot = Aug[j, j]
            Aug[j, :] = Aug[j, :] / pivot
            
            # Eliminar la variable en la columna j de las otras filas
            for k in range(n):
                if k != j:
                    factor = Aug[k, j]
                    Aug[k, :] = Aug[k, :] - factor * Aug[j, :]
        
        # La última columna de la matriz aumentada es la solución para este e_i
        A_inv[:, i] = Aug[:, -1]
    
    return A_inv

## Ejemplos
* Ejemplo 1

In [51]:
# La matriz A =
A = [
    [1, 2, 3, 4],
    [2, 5, 6, 7],
    [3, 6, 8, 9],
    [4, 7, 9, 10],
]
# tiene como inversa
# A_inv =[[ 0.5, -0.5, -1.5,  1.5],
#        [-0.5,  1.5, -1.5,  0.5],
#        [-1.5, -1.5,  3.5, -1.5],
#        [ 1.5,  0.5, -1.5,  0.5]]
inv_matrix(A)

array([[ 0.5, -0.5, -1.5,  1.5],
       [-0.5,  1.5, -1.5,  0.5],
       [-1.5, -1.5,  3.5, -1.5],
       [ 1.5,  0.5, -1.5,  0.5]])

* Ejemplo 2

In [52]:
# La matriz A =
A = [
    [4, 4, 5, 1],
    [3, 4, 2, 2],
    [2, 1, 4, 1],
    [3, 2, 5, 4],
]
# tiene como inversa
# A_inv =[[-34.,  31.,  52., -20.],
#         [ 19., -17., -29.,  11.],
#         [ 12., -11., -18.,   7.],
#         [  1.,  -1.,  -2.,   1.]]
inv_matrix(A)

array([[-34.,  31.,  52., -20.],
       [ 19., -17., -29.,  11.],
       [ 12., -11., -18.,   7.],
       [  1.,  -1.,  -2.,   1.]])

## Ejercicios

* Ejercicio 1

In [53]:
A = [[2, -3], [-1, 1]]
inv_matrix(A)

array([[-1., -3.],
       [-1., -2.]])

* Ejercicio 2

In [54]:
A = [
    [4, 0, 0, 5],
    [1, 0, 4, 0],
    [3, 4, 1, 3],
    [1, 3, 3, 0],
]
inv_matrix(A)

array([[-36.,  45.,  60., -80.],
       [  3.,  -4.,  -5.,   7.],
       [  9., -11., -15.,  20.],
       [ 29., -36., -48.,  64.]])

* Ejercicio 3

In [55]:
A = [
    [0, 0, 0, 0, 0, 0, 1, -1],
    [0, 1, -1, 1, 0, -1, 0, 1],
    [-1, -1, 0, 0, 2, 1, 0, 0],
    [-1, -1, -1, 1, 2, 0, 0, 1],
    [-1, 1, 1, 0, -1, -1, 0, 2],
    [0, 1, 0, 0, -1, -1, 0, 0],
    [1, -1, -1, 1, 2, 1, 0, 2],
    [2, 0, 0, 0, 0, 1, 2, 0],
]
inv_matrix(A)

array([[ 2., -1.,  0., -1.,  0.,  2.,  2., -1.],
       [ 0.,  1.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 6., -1.,  0., -3.,  1.,  1.,  4., -3.],
       [ 6.,  1., -1., -3.,  1., -3.,  3., -3.],
       [ 2., -1.,  1., -1.,  0.,  3.,  2., -1.],
       [-2.,  2., -0.,  0., -0., -4., -2.,  1.],
       [-1., -0., -0.,  1., -0., -0., -1.,  1.],
       [-2., -0., -0.,  1., -0., -0., -1.,  1.]])

* Ejercicio 4

In [56]:
A = [
    [1, 0, 0, 0, -1, 0, 0, -1, 1, -1],
    [1, 1, 0, -1, -1, 1, 0, 0, 1, -1],
    [-1, 0, -1, 0, 0, 0, -1, 1, 0, 0],
    [0, 0, -1, 0, -1, -1, 1, 0, 1, 0],
    [-1, 0, 0, -1, 1, 1, 1, 1, 0, -1],
    [1, 0, 0, 1, -1, -1, -1, 1, -1, 0],
    [1, 1, 1, 0, 1, 0, -1, -1, -1, 1],
    [1, 1, 1, 1, 0, 0, 1, 1, 0, 0],
    [1, 1, 1, 1, 1, 0, -1, -1, 0, 0],
    [0, 0, -1, -1, -1, 0, 1, 1, 1, -1],
]
inv_matrix(A)

array([[ 14.,  -8.,   9.,  -4.,   0.,  -4.,   9.,   7.,  -8.,   3.],
       [ -2.,   2.,  -1.,   2.,   1.,   1.,  -1.,  -1.,   1.,  -2.],
       [-27.,  14., -18.,   5.,  -2.,   7., -17., -13.,  16.,  -2.],
       [ 12.,  -6.,   8.,  -2.,   1.,  -3.,   7.,   6.,  -7.,   0.],
       [  6.,  -4.,   4.,  -2.,   0.,  -2.,   4.,   3.,  -3.,   2.],
       [ 18.,  -9.,  12.,  -4.,   1.,  -5.,  11.,   9., -11.,   1.],
       [  8.,  -4.,   5.,  -1.,   1.,  -2.,   5.,   4.,  -5.,   0.],
       [ -5.,   2.,  -3.,   0.,  -1.,   1.,  -3.,  -2.,   3.,   1.],
       [-11.,   5.,  -7.,   1.,  -2.,   2.,  -7.,  -5.,   7.,   1.],
       [  1.,  -1.,   1.,  -1.,  -1.,  -1.,   1.,   1.,  -1.,   1.]])